<a href="https://colab.research.google.com/github/Anjanisinghthakur/Google-Colab/blob/main/Pyspark_DF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pwd

'/content'

In [ ]:
pip install pyspark

In [ ]:
#if else
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import when, spark_partition_id, broadcast

spark = SparkSession.builder.appName("repartitioning+Joins+cascading_droadcast join").getOrCreate()
employees = [
    ["John Doe", 30, "Software Engineer", 60000],
    ["Jane Smith", 28, "Software Engineer", 55000],
    ["Mike baby", 35, "Data Analyst", 75000],
    ["Harry Potter", 38, "Data Analyst", 75000],
    ["joe goe", 25, "Project Manager", 75000],
    ["cue be", 45, "", 75000]
]

employee_schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Position", StringType(), True),
    StructField("Salary", IntegerType(), True)
])
df=spark.createDataFrame(employees,schema=employee_schema)
df2=df.withColumn("Pos", when(df.Position=="","None").otherwise(df.Position))
df.fillna(0)


DataFrame[Name: string, Age: int, Position: string, Salary: int]

In [ ]:
# repartitioning + coalesce +spark_partition_id
df=df.repartition(4,df.Position)
df=df.coalesce(2)
df.rdd.getNumPartitions()
emp1=df.repartition(4,df.Position).withColumn("Partitionnum",spark_partition_id())
emp1.show()

+------------+---+-----------------+------+------------+
|        Name|Age|         Position|Salary|Partitionnum|
+------------+---+-----------------+------+------------+
|   Mike baby| 35|     Data Analyst| 75000|           0|
|Harry Potter| 38|     Data Analyst| 75000|           0|
|     joe goe| 25|  Project Manager| 75000|           0|
|      cue be| 45|                 | 75000|           0|
|    John Doe| 30|Software Engineer| 60000|           2|
|  Jane Smith| 28|Software Engineer| 55000|           2|
+------------+---+-----------------+------+------------+



In [ ]:

#Joins + cascading(extra conditions) joins
#for and condition use & and for or condition use |

emp_list = [
    [101, "Alice", 1],
    [102, "Bob", 2],
    [103, "Charlie", 1],
    [104, "David", 3],
    [105, "Happy", 4],
    [106, "Cute", 5],
    [108, "Love", 2],
    [109, "Kiki", 2],
    [110, "Doe", 2],


]

dept_list = [
    [1, "HR"],
    [2, "Finance"],
    [3, "Marketing"],
    [4, "IT"]
]
emp_schema = "emp_id int, name string, dept_id int"
dept_schema = "dept_id int, dept_name string"

emp=spark.createDataFrame(emp_list,schema=emp_schema)
dept=spark.createDataFrame(dept_list,schema=dept_schema)
emp.show()
dept.show()
joined=emp.join(broadcast(dept),how="inner",on=(emp.dept_id==dept.dept_id) & ((emp.dept_id==1) | (emp.dept_id==2))
& (emp.name!="")
                )

join1=emp.join(dept, how="cross")
join1.show()
joined.select(emp.name,dept.dept_id,dept.dept_name).show()
joined.explain()

+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|   101|  Alice|      1|
|   102|    Bob|      2|
|   103|Charlie|      1|
|   104|  David|      3|
|   105|  Happy|      4|
|   106|   Cute|      5|
|   108|   Love|      2|
|   109|   Kiki|      2|
|   110|    Doe|      2|
+------+-------+-------+

+-------+---------+
|dept_id|dept_name|
+-------+---------+
|      1|       HR|
|      2|  Finance|
|      3|Marketing|
|      4|       IT|
+-------+---------+

+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|   101|  Alice|      1|      1|       HR|
|   101|  Alice|      1|      2|  Finance|
|   102|    Bob|      2|      1|       HR|
|   102|    Bob|      2|      2|  Finance|
|   103|Charlie|      1|      1|       HR|
|   103|Charlie|      1|      2|  Finance|
|   104|  David|      3|      1|       HR|
|   104|  David|      3|      2|  Finance|
|   101|  Alice|      1|      3|Marketin

In [ ]:
read_csv_df=spark.read.csv("/content/employees.csv",header=True)
read_csv_df.show()
read_csv_df.printSchema()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [ ]:
emp.join(broadcast(dept),how="inner",on=(emp.dept_id==dept.dept_id) & ((emp.dept_id==1) | (emp.dept_id==2))

SyntaxError: incomplete input (<ipython-input-12-e092f684889c>, line 1)

In [ ]:
from pyspark.sql.functions import when
df=spark.read.csv("/content/employees.csv",header=True, inferSchema=True)
df.show()
df.printSchema()
df.withColumn("New",when(df.JOB_ID=="","Outsider").otherwise(df.JOB_ID)).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-Jan-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-Feb-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-Aug-05|    MK_REP|  6000|            - |       201|           20|
